In [ ]:
import csv
import cv2
import os

# Variables globales
image = None
points = []
midpoint = None

def mouse_callback(event, x, y, flags, param):
    global image, points, midpoint
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(points) < 2:
            points.append((x, y))
            cv2.circle(image, (x, y), 3, (0, 255, 0), -1)

            if len(points) == 2:
                calcular_medio()

            cv2.imshow('Image', image)

def calcular_medio():
    global points, midpoint, image
    x1, y1 = points[0]
    x2, y2 = points[1]
    mid_x = (x1 + x2) // 2
    mid_y = (y1 + y2) // 2
    midpoint = (mid_x, mid_y)
    cv2.circle(image, midpoint, 3, (0, 0, 255), -1)
    print(f"Medio: {midpoint}")



In [ ]:
def save_to_csv(data, csv_file):
    """
    Guarda los datos en un archivo CSV.
    """
    file_exists = os.path.exists(csv_file)
    with open(csv_file, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)
    print(f"Datos guardados en {csv_file}")
    


In [ ]:
def process_images_in_directory(directory_path,csv_file='escala.csv'):
    """
    Procesa todas las imágenes en un directorio para calcular y guardar la escala.
    """
    for image_name in os.listdir(directory_path):
        image_path = os.path.join(directory_path, image_name)
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            print(f"Procesando imagen: {image_name}")
            image = cv2.imread(image_path)
            if image is None:
                print(f"No se pudo cargar la imagen {image_name}.")
                continue

            punto_referencia = calcular_medio()

            if punto_referencia is None:
                print(f"No se pudo calcular la longitud en píxeles para {image_name}.")
                continue

            print(f"El punto de referencia es: {punto_referencia}.")
        

            data = {
                'imagen': image_name,
                'punto_referencia': punto_referencia,
            }
            save_to_csv(data, csv_file)

# Configuración inicial
directory_path = '../imagenes/seleccionadas/medio'
csv_file = 'punto_referencia.csv'

# Procesar todas las imágenes en el directorio
process_images_in_directory(directory_path,csv_file)

In [ ]:
import cv2
import os
import csv

# Variables globales
image = None
points = []
midpoint = None

def mouse_callback(event, x, y, flags, param):
    global image, points
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(points) < 2:
            points.append((x, y))
            cv2.circle(image, (x, y), 3, (0, 255, 0), -1)
            cv2.imshow('Image', image)

def calcular_medio():
    global points, midpoint, image
    if len(points) != 2:
        return None
    x1, y1 = points[0]
    x2, y2 = points[1]
    mid_x = (x1 + x2) // 2
    mid_y = (y1 + y2) // 2
    midpoint = (mid_x, mid_y)
    cv2.circle(image, midpoint, 3, (0, 0, 255), -1)
    cv2.imshow('Image', image)
    return midpoint

def save_to_csv(data, csv_file):
    file_exists = os.path.isfile(csv_file)
    with open(csv_file, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def process_images_in_directory(directory_path, csv_file='punto_referencia.csv'):
    global image, points, midpoint

    for image_name in os.listdir(directory_path):
        image_path = os.path.join(directory_path, image_name)
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            print(f"Procesando imagen: {image_name}")
            image = cv2.imread(image_path)
            if image is None:
                print(f"No se pudo cargar la imagen {image_name}.")
                continue

            cv2.namedWindow('Image')
            cv2.setMouseCallback('Image', mouse_callback)

            points = []
            midpoint = None

            while True:
                cv2.imshow('Image', image)
                key = cv2.waitKey(1) & 0xFF

                if key == ord('s'):  # Presione 's' para guardar
                    punto_referencia = calcular_medio()
                    if punto_referencia is not None:
                        print(f"El punto de referencia es: {punto_referencia}.")
                        data = {
                            'imagen': image_name,
                            'punto_referencia_x': punto_referencia[0],
                            'punto_referencia_y': punto_referencia[1],
                        }
                        save_to_csv(data, csv_file)
                        break
                    else:
                        print("Por favor, seleccione dos puntos antes de guardar.")
                elif key == ord('q'):  # Presione 'q' para saltar esta imagen
                    break

            cv2.destroyAllWindows()

# Configuración inicial
directory_path = '../imagenes/seleccionadas/final'
csv_file = 'punto_referencia.csv'

# Procesar todas las imágenes en el directorio
process_images_in_directory(directory_path, csv_file)